In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical


result = pd.read_csv('sample/preprocessed_dataset.csv')
print(result)
data = result.drop('파일명',axis=1)
data = data.drop('선종', axis=1)
data = data.drop('상태', axis=1)
data = data.drop('위치' , axis=1)
print(data)

       파일명   선종    상태                               위치  state  vessel_type  \
0    1.jpg  범장망    표류    LAT N33.22.872 LON E124.5.074      3            4   
1    2.jpg  범장망    이동   LAT N32.48.241 LON E124.38.553      2            4   
2    3.jpg  범장망    조업   LAT N32.55.893 LON E124.33.731      1            4   
3    4.jpg   타망    이동   LAT N35.40.891 LON E124.25.481      2            1   
4    5.jpg   타망    이동   LAT N32.23.668 LON E125.33.053      2            1   
5    6.jpg   타망    이동   LAT N32.59.352 LON E124.29.609      2            1   
6    7.jpg   유망    조업   LAT N34.54.232 LON E124.42.394      1            2   
7    8.jpg   유망    조업   LAT N34.41.829 LON E124.20.120      1            2   
8    9.jpg   유망    표류   LAT N36.56.279 LON E124.39.427      3            2   
9   10.jpg  안강망    조업   LAT N34.15.543 LON E125.41.175      1            5   
10  11.jpg   통발  \t이동   LAT N34.17.855 LON E124.15.509      2            7   
11  12.jpg   통발    이동   LAT N33.41.223 LON E124.12.877      2   

In [10]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical

# 이미지 디렉토리 경로
image_dir = 'image'  # 경로 수정 필요

# 이미지 읽기 및 전처리
images = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

# 레이블을 정의 (예: 0 또는 1)
# 레이블을 적절히 설정하세요
labels = data['vessel_type']
labels = to_categorical(labels, num_classes=10)



In [12]:
train_data = pd.read_csv('preprocessed_whole_dataset.csv')
train_data
target = train_data['vessel_type']
target = to_categorical(target, num_classes=10)  
target

/home/elicer/.local/lib/python3.10/site-packages/pandas/core/series.py:1031: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


IndexError: index 10 is out of bounds for axis 1 with size 10

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def MobileNetV2(input_shape=(256, 256, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MobileNetV2')
    return model

model = MobileNetV2()
model.summary()



/tmp/ipykernel_53349/3574823562.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
2024-10-18 01:41:10.633444: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 8, 8, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 2,389,130 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
history = model.fit(images, target, epochs=200, verbose=1)
predictions = model.predict(images)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

print(class_predictions)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1556
'y' sizes: 1581


In [ ]:
import folium

# 좌표 데이터 (위도, 경도)
locations = [
    {"name": "차", "lat": 32 + 11/60, "lon": 125 + 25/60},
    {"name": "카", "lat": 33 + 20/60, "lon": 124 + 8/60},
    {"name": "타", "lat": 34 + 0/60, "lon": 124 + 0/60 + 30/3600},
    {"name": "파", "lat": 35 + 0/60, "lon": 124 + 7/60 + 30/3600},
    {"name": "하", "lat": 35 + 30/60, "lon": 124 + 30/60},
    {"name": "거", "lat": 36 + 45/60, "lon": 124 + 30/60},
    {"name": "너", "lat": 37 + 0/60, "lon": 124 + 20/60},
    {"name": "바", "lat": 32 + 11/60, "lon": 126 + 45/60},
]

# 초기 지도의 중심 좌표 (평균 값 사용)
center_lat = sum([loc["lat"] for loc in locations]) / len(locations)
center_lon = sum([loc["lon"] for loc in locations]) / len(locations)

# 지도 생성
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# 각 위치에 마커 추가
for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=loc["name"]
    ).add_to(mymap)

# 지도 저장
mymap.save("map.html")

# 지도를 출력하고 싶다면, 주피터 노트북 환경에서는 다음 코드로 출력 가능
mymap


In [ ]:
pip install pytesseract opencv-python pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
